In [ ]:
from IPython.display import display, HTML
# Inject custom CSS to reduce widget label font size by 25%
display(HTML("""
<style>
    .widget-label { font-size: 75% !important; }
</style>
"""))

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox
from matplotlib.ticker import FormatStrFormatter
from matplotlib.patches import Circle

# Set overall chart font size to 10 points.
plt.rcParams.update({'font.size': 10})

def simulate_model(L_active, L_deferred, L_pensioner, 
                   D_active, D_deferred, D_pensioner, 
                   gilt_yield, spread, 
                   plan_assets0, plan_outperf,
                   buffer_assets0, buffer_outperf,
                   max_years,
                   profit_delay, profit_hurdle,
                   scis, scasc,
                   benefit_rate, embedded_prudence):
    """
    Run a half-year simulation with Safe Covenant charges, benefit payments,
    and an additional Embedded Prudence Release (EPR) on liabilities.
    
    At time 0, a Safe Covenant Installation Charge (SCIC) is deducted from the buffer:
      SCIC = max(scis × (1.05 × Total Liabilities[0]), 0.4)
    
    At each integer year end (before profit extraction), two deductions occur in order:
      1. A benefit payment is deducted from plan assets:
         Benefit = benefit_rate × (Total Liabilities at that year end)
      2. A Safe Covenant Annual Service Charge (SCASC) is deducted from the buffer:
         SCASC = scasc × (Total Assets at that year end before profit extraction)
    
    After simulation, the original liability curves (computed via exponential decay)
    are adjusted by subtracting cumulative benefits proportionally. Then, an
    additional reduction is applied year on year via the Embedded Prudence Release (EPR).
    For each time step:
         additional_factor = (1 - EPR)^(floor(time))
    and the final liabilities are given by:
         final_liability = adjusted_liability × additional_factor.
    
    Parameters:
      - benefit_rate: expressed as a fraction (e.g. 0.02 for 2%).
      - embedded_prudence: expressed as a percent (e.g. 1.0 for 1%).
    
    Returns:
      times,
      final liabilities (active, deferred, pensioner, total),
      plan, buffer, total_assets,
      cumulative profit extraction,
      cumulative SCIC, cumulative SCASC, cumulative Benefits,
      and separate (non-cumulative) deductions for SCIC, SCASC, and Benefits.
    """
    dt = 0.5
    times = np.arange(0, max_years + dt, dt)
    n = len(times)
    
    # Compute original liabilities by exponential decay.
    liab_act = L_active * np.exp(-times / D_active)
    liab_def = L_deferred * np.exp(-times / D_deferred)
    liab_pen = L_pensioner * np.exp(-times / D_pensioner)
    liab_total = liab_act + liab_def + liab_pen
    
    # Initialize arrays.
    plan = np.zeros(n)
    buffer = np.zeros(n)
    total_assets = np.zeros(n)
    cum_extraction = np.zeros(n)
    cum_scic = np.zeros(n)   # cumulative SCIC (applied at time 0)
    cum_scasc = np.zeros(n)  # cumulative SCASC (builds each year)
    cum_benefit = np.zeros(n)  # cumulative benefit payments
    
    ded_scic = np.zeros(n)
    ded_scasc = np.zeros(n)
    ded_benefit = np.zeros(n)
    
    plan[0] = plan_assets0
    buffer[0] = buffer_assets0
    total_assets[0] = plan[0] + buffer[0]
    cum_extraction[0] = 0
    
    # SCIC at time 0.
    scic_val = scis * (1.05 * liab_total[0])
    if scic_val < 0.4:
        scic_val = 0.4
    buffer[0] -= scic_val
    cum_scic[0] = scic_val
    ded_scic[0] = scic_val
    cum_scasc[0] = 0
    ded_scasc[0] = 0
    cum_benefit[0] = 0
    ded_benefit[0] = 0
    
    # Simulation loop.
    for i in range(1, n):
        plan[i] = plan[i-1] * np.exp((gilt_yield + plan_outperf) * dt)
        buffer[i] = buffer[i-1] * np.exp((gilt_yield + buffer_outperf) * dt)
        
        # At each integer year end, deduct Benefit from plan assets.
        if np.isclose(times[i] % 1, 0, atol=1e-9):
            benefit_val = benefit_rate * liab_total[i]
            plan[i] -= benefit_val
            ded_benefit[i] = benefit_val
            cum_benefit[i] = cum_benefit[i-1] + benefit_val
        else:
            cum_benefit[i] = cum_benefit[i-1]
            ded_benefit[i] = 0
        
        total_assets[i] = plan[i] + buffer[i]
        
        # Deduct SCASC from buffer at each integer year end.
        if np.isclose(times[i] % 1, 0, atol=1e-9):
            scasc_val = scasc * total_assets[i]
            buffer[i] -= scasc_val
            ded_scasc[i] = scasc_val
            cum_scasc[i] = cum_scasc[i-1] + scasc_val
        else:
            cum_scasc[i] = cum_scasc[i-1]
            ded_scasc[i] = 0
        
        # Profit extraction.
        extraction_tests = 0
        if times[i] >= profit_delay:
            extraction_tests += 1
            if np.isclose(times[i] % 1, 0, atol=1e-9):
                extraction_tests += 1
        for _ in range(extraction_tests):
            tot = plan[i] + buffer[i]
            funding = 100 * tot / liab_total[i]
            if funding > profit_hurdle:
                Y_extracted = tot - (profit_hurdle / 100) * liab_total[i]
                Y = min(Y_extracted, buffer[i])
                buffer[i] -= Y
                cum_extraction[i] += Y
                tot = plan[i] + buffer[i]
        cum_extraction[i] += cum_extraction[i-1]
        total_assets[i] = plan[i] + buffer[i]
        cum_scic[i] = cum_scic[0]
        ded_scic[i] = 0
    
    # Adjust liabilities by subtracting cumulative benefits proportionally.
    adj_liab_total = np.maximum(liab_total - cum_benefit, 0)
    factor = np.where(liab_total > 0, adj_liab_total / liab_total, 0)
    adj_liab_act = liab_act * factor
    adj_liab_def = liab_def * factor
    adj_liab_pen = liab_pen * factor
    
    # Apply Embedded Prudence Release (EPR)
    epr = embedded_prudence / 100.0  # convert percent to fraction
    additional_factor = np.power(1 - epr, np.floor(times))
    final_liab_act = adj_liab_act * additional_factor
    final_liab_def = adj_liab_def * additional_factor
    final_liab_pen = adj_liab_pen * additional_factor
    final_liab_total = final_liab_act + final_liab_def + final_liab_pen
    
    return (times, final_liab_act, final_liab_def, final_liab_pen, final_liab_total, 
            plan, buffer, total_assets, cum_extraction, cum_scic, cum_scasc, cum_benefit,
            ded_scic, ded_scasc, ded_benefit)

def plot_model_combo(L_active, L_deferred, L_pensioner, 
                     D_active, D_deferred, D_pensioner, 
                     gilt_yield, spread, 
                     plan_assets0, plan_outperf,
                     buffer_assets0, buffer_outperf,
                     max_years,
                     profit_delay, profit_hurdle,
                     scis, scasc, benefit_rate, embedded_prudence,
                     show_active, show_deferred, show_pensioner, show_total_liability,
                     show_plan_assets, show_buffer_assets, show_total_assets,
                     show_plan_funding, show_total_funding,
                     add_profit_line,
                     plot_3D,
                     show_scic, show_scasc,
                     show_cum_benefit, separate_scic, separate_scasc, separate_benefit,
                     elev, azim):
    """
    Generates either a 3D or 2D plot of adjusted liabilities, assets, and 
    separate/cumulative charges (SCIC, SCASC, Benefits).
    
    New parameters:
      - scis: SCIS Level.
      - scasc: SCASC Level.
      - benefit_rate: Annual benefit rate (Benefit = benefit_rate × Liabilities at year end).
      - embedded_prudence: Embedded Prudence Release percentage (applied year-on-year).
      - show_scic: if True, display cumulative SCIC as a line.
      - show_scasc: if True, display cumulative SCASC as a line.
      - show_cum_benefit: if True, display cumulative benefits as a line.
      - separate_scic, separate_scasc, separate_benefit: display separate (non-cumulative) bars.
    """
    (times, liab_act, liab_def, liab_pen, liab_total, 
     plan, buffer, total_assets, cum_extraction, cum_scic, cum_scasc, cum_benefit,
     ded_scic, ded_scasc, ded_benefit) = simulate_model(
         L_active, L_deferred, L_pensioner, 
         D_active, D_deferred, D_pensioner, 
         gilt_yield, spread, 
         plan_assets0, plan_outperf,
         buffer_assets0, buffer_outperf,
         max_years,
         profit_delay, profit_hurdle,
         scis, scasc, benefit_rate, embedded_prudence)
    
    # Funding levels (using final liabilities)
    funding_plan = 100 * plan / liab_total
    funding_total = 100 * total_assets / liab_total
    
    if plot_3D:
        fig = plt.figure(figsize=(10.8, 7.2))
        ax = fig.add_subplot(111, projection='3d')
        if show_active:
            ax.plot(times, liab_act, zs=0, zdir='z', label='Actives Liability', color='red')
        if show_deferred:
            ax.plot(times, liab_def, zs=0, zdir='z', label='Deferred Liability', color='orange')
        if show_pensioner:
            ax.plot(times, liab_pen, zs=0, zdir='z', label='Pensioners Liability', color='brown')
        if show_total_liability:
            ax.plot(times, liab_total, zs=0, zdir='z', label='Total Liability', color='black', linestyle='--', linewidth=2)
        
        if show_plan_assets:
            ax.plot(times, plan, zs=0, zdir='z', label='Plan Assets (£m)', color='green')
        if show_buffer_assets:
            ax.plot(times, buffer, zs=0, zdir='z', label='Buffer Assets (£m)', color='blue')
        if show_total_assets:
            ax.plot(times, total_assets, zs=0, zdir='z', label='Total Assets (£m)', color='purple', linestyle='--', linewidth=2)
        
        if show_plan_funding:
            ax.plot(times, funding_plan, zs=0, zdir='z', label='Plan Funding (%)', color='green', linestyle=':')
        if show_total_funding:
            ax.plot(times, funding_total, zs=0, zdir='z', label='Total Funding (%)', color='purple', linestyle=':')
        
        ax.plot(times, np.zeros_like(times), cum_extraction, label='Cumulative Profit Extracted (£m)', color='magenta', linewidth=2)
        if show_scic:
            ax.plot(times, np.full_like(times, cum_scic[0]), label='Cumulative SCIC (£m)', color='cyan', linestyle='--', linewidth=2)
        if show_scasc:
            ax.plot(times, cum_scasc, label='Cumulative SCASC (£m)', color='orange', linestyle='--', linewidth=2)
        if show_cum_benefit:
            ax.plot(times, cum_benefit, label='Cumulative Benefits (£m)', color='blue', linestyle='--', linewidth=2)
        
        if separate_scic:
            ax.bar3d(0, 0, 0, 0.2, 0.2, ded_scic[0], color='cyan', alpha=0.7, label='Separate SCIC (£m)')
        if separate_scasc:
            integer_indices = np.where(np.isclose(times % 1, 0, atol=1e-9))[0]
            first = True
            for idx in integer_indices:
                label = 'Separate SCASC (£m)' if first else "_nolegend_"
                first = False
                ax.bar3d(times[idx], 0, 0, 0.2, 0.2, ded_scasc[idx], color='orange', alpha=0.7, label=label)
        if separate_benefit:
            integer_indices = np.where(np.isclose(times % 1, 0, atol=1e-9))[0]
            first = True
            for idx in integer_indices:
                label = 'Separate Benefits (£m)' if first else "_nolegend_"
                first = False
                ax.bar3d(times[idx], 0, 0, 0.2, 0.2, ded_benefit[idx], color='magenta', alpha=0.7, label=label)
        
        ax.set_xlabel('Time (years)')
        ax.set_ylabel('Value (£m)')
        ax.set_zlabel('Cumulative Amount (£m)')
        ax.set_title('3D View: Liabilities, Assets & Charges\n(All amounts in GBP £ millions)')
        ax.view_init(elev=elev, azim=azim)
        ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
        plt.show()
    else:
        fig, ax1 = plt.subplots(figsize=(9, 5.4))
        
        if show_active:
            ax1.plot(times, liab_act, label='Actives Liability', color='red')
        if show_deferred:
            ax1.plot(times, liab_def, label='Deferred Liability', color='orange')
        if show_pensioner:
            ax1.plot(times, liab_pen, label='Pensioners Liability', color='brown')
        if show_total_liability:
            ax1.plot(times, liab_total, label='Total Liability', color='black', linestyle='--', linewidth=2)
        
        if show_plan_assets:
            ax1.plot(times, plan, label='Plan Assets (£m)', color='green')
        if show_buffer_assets:
            ax1.plot(times, buffer, label='Buffer Assets (£m)', color='blue')
        if show_total_assets:
            ax1.plot(times, total_assets, label='Total Assets (£m)', color='purple', linestyle='--', linewidth=2)
        
        if add_profit_line:
            ax1.plot(times, cum_extraction, label='Cumulative Profit Extracted (£m)', color='magenta', linewidth=2)
        if show_scic:
            ax1.plot(times, np.full_like(times, cum_scic[0]), label='Cumulative SCIC (£m)', color='cyan', linestyle='--', linewidth=2)
        if show_scasc:
            ax1.step(times, cum_scasc, where='pre', label='Cumulative SCASC (£m)', color='orange', linestyle='--', linewidth=2)
        if show_cum_benefit:
            ax1.plot(times, cum_benefit, label='Cumulative Benefits (£m)', color='blue', linestyle='--', linewidth=2)
        
        if separate_scic:
            ax1.bar(0, ded_scic[0], width=0.1, color='cyan', alpha=0.7, label='Separate SCIC (£m)')
        if separate_scasc:
            integer_indices = np.where(np.isclose(times % 1, 0, atol=1e-9))[0]
            ax1.bar(times[integer_indices], ded_scasc[integer_indices], width=0.1, color='orange', alpha=0.7, label='Separate SCASC (£m)')
        if separate_benefit:
            integer_indices = np.where(np.isclose(times % 1, 0, atol=1e-9))[0]
            ax1.bar(times[integer_indices], ded_benefit[integer_indices], width=0.1, color='magenta', alpha=0.7, label='Separate Benefits (£m)')
        
        ax1.set_xlabel('Time (years)')
        ax1.set_ylabel('Value (£m)')
        ax1.set_title('2D View: Liabilities, Assets & Charges\n(All amounts in GBP £ millions)')
        ax1.grid(True)
        
        if show_plan_funding or show_total_funding:
            ax2 = ax1.twinx()
            if show_plan_funding:
                ax2.plot(times, funding_plan, label='Plan Funding (%)', color='green', linestyle=':')
            if show_total_funding:
                ax2.plot(times, funding_total, label='Total Funding (%)', color='purple', linestyle=':')
            ax2.set_ylabel('Funding Level (%)')
            ax2.yaxis.set_major_formatter(FormatStrFormatter('%d'))
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
        else:
            ax1.legend(loc='upper right')
        
        inset_size = 1.97 / 9  
        inset_left = 0.40     
        inset_bottom = -0.15  
        ax_zoom = fig.add_axes([inset_left, inset_bottom, inset_size, inset_size])
        circle = Circle((0.5, 0.5), 0.5, transform=ax_zoom.transAxes, fc='none', ec='black', lw=1.5)
        ax_zoom.set_clip_path(circle)
        ax_zoom.set_facecolor('white')
        ax_zoom.set_xticks([])
        ax_zoom.set_yticks([])
        ax_zoom.set_visible(False)
        
        def on_move(event):
            if event.inaxes == ax1 and event.xdata is not None and event.ydata is not None:
                xlim = ax1.get_xlim()
                ylim = ax1.get_ylim()
                dx = (xlim[1] - xlim[0]) / 20.0
                dy = (ylim[1] - ylim[0]) / 20.0
                ax_zoom.set_xlim(event.xdata - dx/2, event.xdata + dx/2)
                ax_zoom.set_ylim(event.ydata - dy/2, event.ydata + dy/2)
                ax_zoom.set_visible(True)
            else:
                ax_zoom.set_visible(False)
            fig.canvas.draw_idle()
        
        fig.canvas.mpl_connect('motion_notify_event', on_move)
        plt.show()

# -------------------------
# Define widget controls
# -------------------------
L_active_w = widgets.FloatSlider(min=0, max=1000, step=10, value=0, description='Active Liability (£m)')
L_deferred_w = widgets.FloatSlider(min=0, max=1000, step=10, value=300, description='Deferred Liability (£m)')
L_pensioner_w = widgets.FloatSlider(min=0, max=1000, step=10, value=700, description='Pensioner Liability (£m)')
D_active_w = widgets.IntSlider(min=1, max=40, step=1, value=20, description='Active Duration (years)')
D_deferred_w = widgets.IntSlider(min=1, max=40, step=1, value=15, description='Deferred Duration (years)')
D_pensioner_w = widgets.IntSlider(min=1, max=40, step=1, value=10, description='Pensioner Duration (years)')
gilt_yield_w = widgets.FloatSlider(min=0.0, max=0.10, step=0.001, value=0.045, description='Gilt Yield', readout_format='.3f')
spread_w = widgets.FloatSlider(min=0.0, max=0.10, step=0.001, value=0.005, description='Spread', readout_format='.3f')
plan_assets0_w = widgets.FloatSlider(min=0, max=2000, step=10, value=950, description='Plan Assets (£m)')
plan_outperf_w = widgets.FloatSlider(min=0.0, max=0.10, step=0.001, value=0.01, description='Plan Outperf', readout_format='.3f')
buffer_assets0_w = widgets.FloatSlider(min=0, max=2000, step=10, value=200, description='Buffer Assets (£m)')
buffer_outperf_w = widgets.FloatSlider(min=0.0, max=0.10, step=0.001, value=0.02, description='Buffer Outperf', readout_format='.3f')
# Set initial Time Horizon to 19.
max_years_w = widgets.IntSlider(min=1, max=40, step=1, value=19, description='Time Horizon (years)')
profit_delay_w = widgets.FloatSlider(min=0, max=20, step=0.5, value=0, description='Profit Delay (Z, years)')
profit_hurdle_w = widgets.IntSlider(min=80, max=150, step=1, value=120, description='Profit Hurdle (X %)')
scis_w = widgets.FloatSlider(min=0.00000, max=0.001, step=0.000025, value=0.00075, description='SCIS Level', readout_format='.5f')
scasc_w = widgets.FloatSlider(min=0.00000, max=0.0075, step=0.00005, value=0.0015, description='SCASC Level', readout_format='.5f')
benefit_rate_w = widgets.FloatSlider(min=0.0, max=0.1, step=0.001, value=0.02, description='Benefit Rate', readout_format='.3f')
# Embedded Prudence Release slider: range 0% to 3%, step 0.005%, initial value 1%.
embedded_prudence_w = widgets.FloatSlider(min=0.0, max=3.0, step=0.005, value=1.0, description='Embedded Prudence (%)', readout_format='.3f')

show_scic_w = widgets.Checkbox(value=False, description='Show Cumulative SCIC')
show_scasc_w = widgets.Checkbox(value=False, description='Show Cumulative SCASC')
show_cum_benefit_w = widgets.Checkbox(value=False, description='Show Cumulative Benefits')
separate_scic_w = widgets.Checkbox(value=False, description='Show Separate SCIC')
separate_scasc_w = widgets.Checkbox(value=False, description='Show Separate SCASC')
separate_benefit_w = widgets.Checkbox(value=False, description='Show Separate Benefits')

show_active_w = widgets.Checkbox(value=False, description='Actives Liability')
show_deferred_w = widgets.Checkbox(value=False, description='Deferred Liability')
show_pensioner_w = widgets.Checkbox(value=False, description='Pensioners Liability')
show_total_liability_w = widgets.Checkbox(value=True, description='Total Liability')
show_plan_assets_w = widgets.Checkbox(value=False, description='Plan Assets')
show_buffer_assets_w = widgets.Checkbox(value=False, description='Buffer Assets')
show_total_assets_w = widgets.Checkbox(value=True, description='Total Assets')
show_plan_funding_w = widgets.Checkbox(value=False, description='Plan Funding Level')
show_total_funding_w = widgets.Checkbox(value=True, description='Total Funding Level')
add_profit_line_w = widgets.Checkbox(value=False, description='Add profit extraction line?')
plot_3D_w = widgets.Checkbox(value=False, description='Plot in 3D?')
elev_w = widgets.IntSlider(min=-90, max=90, step=5, value=35, description='Elevation')
azim_w = widgets.IntSlider(min=-180, max=180, step=5, value=-105, description='Azimuth')

controls = VBox([
    L_active_w, L_deferred_w, L_pensioner_w,
    D_active_w, D_deferred_w, D_pensioner_w,
    gilt_yield_w, spread_w,
    plan_assets0_w, plan_outperf_w,
    buffer_assets0_w, buffer_outperf_w,
    max_years_w, profit_delay_w, profit_hurdle_w,
    scis_w, scasc_w, benefit_rate_w, embedded_prudence_w,
    show_scic_w, show_scasc_w, show_cum_benefit_w,
    separate_scic_w, separate_scasc_w, separate_benefit_w,
    show_active_w, show_deferred_w, show_pensioner_w, show_total_liability_w,
    show_plan_assets_w, show_buffer_assets_w, show_total_assets_w,
    show_plan_funding_w, show_total_funding_w,
    add_profit_line_w, plot_3D_w,
    elev_w, azim_w
])

out = interactive_output(plot_model_combo, {
    'L_active': L_active_w,
    'L_deferred': L_deferred_w,
    'L_pensioner': L_pensioner_w,
    'D_active': D_active_w,
    'D_deferred': D_deferred_w,
    'D_pensioner': D_pensioner_w,
    'gilt_yield': gilt_yield_w,
    'spread': spread_w,
    'plan_assets0': plan_assets0_w,
    'plan_outperf': plan_outperf_w,
    'buffer_assets0': buffer_assets0_w,
    'buffer_outperf': buffer_outperf_w,
    'max_years': max_years_w,
    'profit_delay': profit_delay_w,
    'profit_hurdle': profit_hurdle_w,
    'scis': scis_w,
    'scasc': scasc_w,
    'benefit_rate': benefit_rate_w,
    'embedded_prudence': embedded_prudence_w,
    'show_active': show_active_w,
    'show_deferred': show_deferred_w,
    'show_pensioner': show_pensioner_w,
    'show_total_liability': show_total_liability_w,
    'show_plan_assets': show_plan_assets_w,
    'show_buffer_assets': show_buffer_assets_w,
    'show_total_assets': show_total_assets_w,
    'show_plan_funding': show_plan_funding_w,
    'show_total_funding': show_total_funding_w,
    'add_profit_line': add_profit_line_w,
    'plot_3D': plot_3D_w,
    'elev': elev_w,
    'azim': azim_w,
    'show_scic': show_scic_w,
    'show_scasc': show_scasc_w,
    'show_cum_benefit': show_cum_benefit_w,
    'separate_scic': separate_scic_w,
    'separate_scasc': separate_scasc_w,
    'separate_benefit': separate_benefit_w
})

# Arrange the layout so that the sliders are on top and the chart is below.
app = VBox([controls, out])
app
